# Tensorflow ImageNet Patch demo for Securing AI Lab deployment

This notebook contains an end-to-end demostration for the Securing AI Lab Architecture when it is deployed on the DGX workstation.

## Setup

**Note:** This demo is specifically for the NCCoE DGX Workstation with hostname `dgx-station-2`.

Port forwarding is required in order to run this demo.
The recommended port mapping is as follows:

- Map `localhost:30080` on laptop to `localhost:30080` on `dgx-station-2`
- Map `localhost:35000` on laptop to `localhost:35000` on `dgx-station-2`

A sample SSH config file that enables the above port forwarding is provided below,

> ⚠️ **Edits required**: replace `username` with your assigned username _on the NCCoE virtual machines_!

```conf
# vm hostname: jumphost001
Host nccoe-jumphost001
    Hostname 10.33.53.98
    User username
    Port 54131
    IdentityFile %d/.ssh/nccoe-vm

# vm hostname: dgx-station-2
Host nccoe-k8s-gpu002
    Hostname 192.168.1.28
    User username
    Port 22
    IdentityFile %d/.ssh/nccoe-vm
    ProxyJump nccoe-jumphost001
    LocalForward 30080 localhost:30080
    LocalForward 35000 localhost:35000
```

Now, connect to the NCCoE VPN and SSH into the DGX Workstation,

```bash
ssh nccoe-k8s-gpu002
```

Next, we import the necessary Python modules and ensure the proper environment variables are set so that all the code blocks will work as expected,

In [1]:
# Import packages from the Python standard library
import os
import pprint
import time
import warnings
from pathlib import Path
from typing import Tuple

# Please enter custom username here.
USERNAME = "howard"

# Filter out warning messages
warnings.filterwarnings("ignore")

# Default address for accessing the RESTful API service
RESTAPI_ADDRESS = "http://localhost:30080"

# Base API address
RESTAPI_API_BASE = f"{RESTAPI_ADDRESS}/api"

# Default address for accessing the MLFlow Tracking server
MLFLOW_TRACKING_URI = "http://localhost:35000"

# Path to workflows archive
WORKFLOWS_TAR_GZ = Path("workflows.tar.gz")

# Experiment name (note the username_ prefix convention)
EXPERIMENT_NAME = f"{USERNAME}_imagenet_adversarial_patch"

# Set MLFLOW_TRACKING_URI variable, used to connect to MLFlow Tracking service
if os.getenv("MLFLOW_TRACKING_URI") is None:
    os.environ["MLFLOW_TRACKING_URI"] = MLFLOW_TRACKING_URI

# Import third-party Python packages
import numpy as np
import requests
from mlflow.tracking import MlflowClient

# Import utils.py file
import utils

# Create random number generator
rng = np.random.default_rng(54399264723942495723666216079516778448)

Check that the Makefile works in your environment by executing the `bash` code block below,

In [2]:
%%bash

# Running this will just list the available rules defined in the demo's Makefile.
make

Available rules:

clean               Remove temporary files 
data                Download and prepare MNIST dataset 
initdb              Initialize the RESTful API database 
services            Launch the Minio S3 and MLFlow Tracking services 
teardown            Destroy service containers 
workflows           Create workflows tarball 


The ImageNet dataset needed for the demo is available in all worker containers at the path `/nfs/data/ImageNet-Kaggle-2017/images/ILSVRC/Data/CLS-LOC/`.
The following directores contain labeled JPEG images:

    train  
    val-sorted  
    val-sorted-1000
    val-sorted-5000
    val-sorted-10000  
    
Both train and val-sorted contain a full set of labeled training and test images. val-sorted-# directories hold a random partion of labeled images, with 1000, 5000, or 10000 images in total. 

   Each of the directories listed follows the general subfolder structure below:

    ImageNet
    │ 
    │ 
    ├── train
    │   ├── {class 0 directory}
    │   ├── {class 1 directory}
    │   ├── ...
    │   ├── ...
    │   ├── ...
    │   └── {class 999 directory}
    │ 
    └── val-sorted
    │ 
    │   ├── {class 0 directory}
    │   ├── {class 1 directory}
    │   ├── ...
    │   ├── ...
    │   ├── ...
    │   └── {class 999 directory}

The subfolders under `Mnist/training/` and `Mnist/testing/` are the classification labels for the images in the dataset.
This folder structure is a standardized way to encode the label information and many libraries can make use of it, including the Tensorflow library that we are using for this particular demo.

## Submit and run jobs

The jobs that we will be running are implemented in the Python source files under `src/`, which will be executed using the entrypoints defined in the `MLproject` file.
To get this information into the architecture, we need to package those files up into an archive and upload it to the lab API.
For convenience, the `Makefile` provides a rule for creating the archive file, just run `make workflows`,

In [3]:
%%bash

# Create the workflows.tar.gz file
make workflows

make: Nothing to be done for 'workflows'.


To connect with the endpoint, we will use a client class defined in the `utils.py` file that is able to connect with the lab's RESTful API using the HTTP protocol.
We connect using the client below,

In [4]:
restapi_client = utils.SecuringAIClient(address=RESTAPI_API_BASE)

We need to register an experiment under which to collect our job runs.
The code below checks if the relevant experiment exists.
If it does, then it just returns info about the experiment, if it doesn't, it then registers the new experiment.

In [5]:
response_experiment = restapi_client.get_experiment_by_name(name=EXPERIMENT_NAME)

if response_experiment is None or "Not Found" in response_experiment.get("message", []):
    response_experiment = restapi_client.register_experiment(name=EXPERIMENT_NAME)

response_experiment

{'createdOn': '2021-01-06T20:00:36.372309',
 'experimentId': 20,
 'lastModified': '2021-01-06T20:00:36.372309',
 'name': 'howard_imagenet_adversarial_patch'}

Next, we need to train our model.
We will be using the V100 GPUs that are available on the DGX Workstation, which we can use by submitting our job to the `"tensorflow_gpu"` queue.
We will train the ResNet50 model on a subset of the ImageNet dataset.,

In [6]:
response_resnet50_train = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="init_model",
    entry_point_kwargs=" ".join([
        "-P batch_size=20",
        "-P model_architecture=resnet50",
        "-P data_dir=/nfs/data/ImageNet-Kaggle-2017/images/ILSVRC/Data/CLS-LOC/val-sorted-1000",
    ]),
    queue="tensorflow_gpu",
    timeout="1h",
)

print("Training job for ResNet50 neural network submitted")
print("")
pprint.pprint(response_resnet50_train)

Training job for ResNet50 neural network submitted

{'createdOn': '2021-01-06T21:47:40.839114',
 'dependsOn': None,
 'entryPoint': 'init_model',
 'entryPointKwargs': '-P batch_size=20 -P model_architecture=resnet50 -P '
                     'data_dir=/nfs/data/ImageNet-Kaggle-2017/images/ILSVRC/Data/CLS-LOC/val-sorted-1000',
 'experimentId': 20,
 'jobId': '71faeb56-1ef5-41f5-96f0-dc41e7498826',
 'lastModified': '2021-01-06T21:47:40.839114',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '1h',
 'workflowUri': 's3://workflow/c58b1518290e47d4bf81ac2546efbe40/workflows.tar.gz'}


### Deploying and Testing Adversarial Patches

Now we will apply the adversarial patches over our test set and evaluate the performance of the baseline model on the adversarial patches.

In [7]:
def mlflow_run_id_is_not_known(response_patch):
    return response_patch["mlflowRunId"] is None and response_patch["status"] not in [
        "failed",
        "finished",
    ]

# Create Patches
response_resnet50_patch = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="gen_patch",
    entry_point_kwargs=" ".join(
        [
            f"-P model={EXPERIMENT_NAME}_default_pretrained_resnet50/None",
            "-P data_dir=/nfs/data/ImageNet-Kaggle-2017/images/ILSVRC/Data/CLS-LOC/val-sorted-5000",
            "-P num_patch_gen_samples=10",
            "-P num_patch=1",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_resnet50_train["jobId"],
)

print("Patch attack (ResNet50 architecture) job submitted")
print("")
pprint.pprint(response_resnet50_patch)
print("")

# Wait for training to complete.
while mlflow_run_id_is_not_known(response_resnet50_patch):
    time.sleep(1)
    response_resnet50_patch = restapi_client.get_job_by_id(response_resnet50_patch["jobId"])

Patch attack (ResNet50 architecture) job submitted

{'createdOn': '2021-01-06T21:47:40.894281',
 'dependsOn': '71faeb56-1ef5-41f5-96f0-dc41e7498826',
 'entryPoint': 'gen_patch',
 'entryPointKwargs': '-P '
                     'model=howard_imagenet_adversarial_patch_default_pretrained_resnet50/None '
                     '-P '
                     'data_dir=/nfs/data/ImageNet-Kaggle-2017/images/ILSVRC/Data/CLS-LOC/val-sorted-5000 '
                     '-P num_patch_gen_samples=10 -P num_patch=1',
 'experimentId': 20,
 'jobId': 'ec19a826-72a0-4109-8d97-94a82430197e',
 'lastModified': '2021-01-06T21:47:40.894281',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '24h',
 'workflowUri': 's3://workflow/9e7369fa0bb349c9aa08f31052ad5ddd/workflows.tar.gz'}



In [8]:
# Create Patches
# Deploy Patch attack on test set.
response_deploy_resnet50_patch_testing = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="deploy_patch",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_resnet50_patch['mlflowRunId']}",
            f"-P model={EXPERIMENT_NAME}_default_pretrained_resnet50/None",
            "-P data_dir=/nfs/data/ImageNet-Kaggle-2017/images/ILSVRC/Data/CLS-LOC/val-sorted-5000",
            "-P batch_size=500",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_resnet50_patch["jobId"],
)

print("Patch deployment (ResNet50 architecture) job submitted")
print("")
pprint.pprint(response_deploy_resnet50_patch_testing)
print("")

# Deploy Patches
while mlflow_run_id_is_not_known(response_deploy_resnet50_patch_testing):
    time.sleep(1)
    response_deploy_resnet50_patch_testing = restapi_client.get_job_by_id(response_deploy_resnet50_patch_testing["jobId"])

Patch deployment (ResNet50 architecture) job submitted

{'createdOn': '2021-01-06T21:48:27.681010',
 'dependsOn': 'ec19a826-72a0-4109-8d97-94a82430197e',
 'entryPoint': 'deploy_patch',
 'entryPointKwargs': '-P run_id=21549c8346d9481ea6a50986a7879514 -P '
                     'model=howard_imagenet_adversarial_patch_default_pretrained_resnet50/None '
                     '-P '
                     'data_dir=/nfs/data/ImageNet-Kaggle-2017/images/ILSVRC/Data/CLS-LOC/val-sorted-5000 '
                     '-P batch_size=500',
 'experimentId': 20,
 'jobId': '0346af85-eaf6-4fc7-8b71-a75229d2f12b',
 'lastModified': '2021-01-06T21:48:27.681010',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '24h',
 'workflowUri': 's3://workflow/4188c1f8eed84d22b2cc0bf7636038f0/workflows.tar.gz'}



In [9]:
# Check patched dataset results   
response_resnet50_infer_resnet50_patch = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_deploy_resnet50_patch_testing['mlflowRunId']}",
            f"-P model={EXPERIMENT_NAME}_default_pretrained_resnet50/None",
            "-P model_architecture=resnet50",
            "-P batch_size=100",
            "-P dataset_tar_name=adversarial_patch_dataset.tar.gz",
            "-P dataset_name=adv_patch_dataset",
            
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_deploy_resnet50_patch_testing["jobId"],
)

print("Patch evaluation (ResNet50 architecture) job submitted")
print("")
pprint.pprint(response_deploy_resnet50_patch_testing)
print("")

Patch evaluation (ResNet50 architecture) job submitted

{'createdOn': '2021-01-06T21:48:27.681010',
 'dependsOn': 'ec19a826-72a0-4109-8d97-94a82430197e',
 'entryPoint': 'deploy_patch',
 'entryPointKwargs': '-P run_id=21549c8346d9481ea6a50986a7879514 -P '
                     'model=howard_imagenet_adversarial_patch_default_pretrained_resnet50/None '
                     '-P '
                     'data_dir=/nfs/data/ImageNet-Kaggle-2017/images/ILSVRC/Data/CLS-LOC/val-sorted-5000 '
                     '-P batch_size=500',
 'experimentId': 20,
 'jobId': '0346af85-eaf6-4fc7-8b71-a75229d2f12b',
 'lastModified': '2021-01-06T21:55:53.386637',
 'mlflowRunId': 'd87366114ab141909a809b8c91cad994',
 'queueId': 2,
 'status': 'started',
 'timeout': '24h',
 'workflowUri': 's3://workflow/4188c1f8eed84d22b2cc0bf7636038f0/workflows.tar.gz'}



## Querying the MLFlow Tracking Service

Currently the lab API can only be used to register experiments and start jobs, so if users wish to extract their results programmatically, they can use the `MlflowClient()` class from the `mlflow` Python package to connect and query their results.
Since we captured the run ids generated by MLFlow, we can easily retrieve the data logged about one of our jobs and inspect the results.
To start the client, we simply need to run,

In [10]:
mlflow_client = MlflowClient()

The client uses the environment variable `MLFLOW_TRACKING_URI` to figure out how to connect to the MLFlow Tracking Service, which we configured near the top of this notebook.
To query the results of one of our runs, we just need to pass the run id to the client's `get_run()` method.
As an example, let's query the run results for the patch attack applied to the ResNet50 architecture,

In [11]:
run_resnet50 = mlflow_client.get_run(response_resnet50["mlflowRunId"])

NameError: name 'response_resnet50' is not defined

If the request completed successfully, we should now be able to query data collected during the run.
For example, to review the collected metrics, we just use,

In [ ]:
pprint.pprint(run_resnet50.data.metrics)

To review the run's parameters, we use,

In [ ]:
pprint.pprint(run_resnet50.data.params)

To review the run's tags, we use,

In [ ]:
pprint.pprint(run_resnet50.data.tags)

There are many things you can query using the MLFlow client.
[The MLFlow documentation gives a full overview of the methods that are available](https://www.mlflow.org/docs/latest/python_api/mlflow.tracking.html#mlflow.tracking.MlflowClient).